In [1]:
import pandas as pd

## 사용할 데이터 선언
df_wine = pd.read_csv('https://bit.ly/wine_csv_data')

## 샘플과 타겟 분리
arr_data = df_wine[['alcohol', 'sugar', 'pH']].to_numpy()
arr_target = df_wine['class'].to_numpy()

In [2]:
from sklearn.model_selection import train_test_split

## 훈련 세트, 테스트 세트 나누기
## 매개변수 test_size로 비율 정해주기
arr_train_input, arr_test_input, arr_train_target, arr_test_target = train_test_split(
    arr_data, arr_target, test_size= 0.2, random_state= 42
)

In [3]:
## 훈련 세트를 다시 훈련세트와 검증세트로 나누기
arr_sub_input, arr_val_input, arr_sub_target, arr_val_target = train_test_split(
    arr_train_input, arr_train_target, test_size= 0.2, random_state= 42
)

In [4]:
## 훈련 세트와 검증세트 확인
print(arr_sub_input.shape, arr_val_input.shape)

(4157, 3) (1040, 3)


In [5]:
from sklearn.tree import DecisionTreeClassifier

## 결정트리 객체 생성 후 훈련
c_dt = DecisionTreeClassifier(random_state= 42)
c_dt.fit(arr_sub_input, arr_sub_target)
## 훈련 세트와 검증세트로 정확도 비교
## 확실히 훈련세트에 과대적합 되어있음
print(c_dt.score(arr_sub_input, arr_sub_target))
print(c_dt.score(arr_val_input, arr_val_target))

0.9971133028626413
0.864423076923077


In [7]:
from sklearn.model_selection import cross_validate

## 교차 검증 함수를 사용 후 결과 출력
## 기본적으로 5폴드 교차 검증으로 실행
dict_scores = cross_validate(c_dt, arr_train_input, arr_train_target)
print(dict_scores)

{'fit_time': array([0.00672054, 0.00601292, 0.00600386, 0.00496697, 0.00499701]), 'score_time': array([0.00146985, 0.00099707, 0.00102329, 0.00100279, 0.        ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [8]:
import numpy as np
## 최종 검증 점수 계산
print(np.mean(dict_scores['test_score']))

0.855300214703487


In [9]:
from sklearn.model_selection import StratifiedKFold

## 함수 cross_validate를 사용할 때 분할기로 StratifiedKFold를 사용하기 위해 
## 매개변수 cv에 객체 생성 후 최종 검증 점수 확인
dict_scores = cross_validate(c_dt, arr_train_input, arr_train_target, cv= StratifiedKFold())
print(np.mean(dict_scores['test_score']))

0.855300214703487


In [10]:
## 훈련 세트를 섞고 10폴드 교차 검증을 수행후 최종 검증 점수 확인
c_splitter = StratifiedKFold(n_splits= 10, shuffle= True, random_state= 42)
dict_scores = cross_validate(c_dt, arr_train_input, arr_train_target, cv= c_splitter)
print(np.mean(dict_scores['test_score']))

0.8574181117533719


In [11]:
## 하이퍼 파라미터의 최적값을 찾기 위해 필요한 모듈 import
from sklearn.model_selection import GridSearchCV

## 탐색할 매개변수와 값의 리스트를 딕셔너리로 선언
dict_params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [13]:
## 그리드 서치 객체 생성
## 결정 트리 모델의 하이퍼 파라미터(dict_params 안에 있는 매개변수의)의 값을 검색 
## n_jobs : 사용할 cpu core 의 개수 (-1이면 모든 core 사용)
c_gs = GridSearchCV(DecisionTreeClassifier(random_state= 42), dict_params, n_jobs= -1)

In [14]:
## 훈련 세트로 훈련
## cv 매개변수는 기본값이 5이므로 min_impurity_decrease값 5개를 5번 교차검증을 수행해
## 총 25개의 모델을 검증
c_gs.fit(arr_train_input, arr_train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [15]:
## 25개의 모델중 가장 최적의 모델을 결정정 트리 모델로 선언 후 성능 확인
c_dt = c_gs.best_estimator_
print(c_dt.score(arr_train_input, arr_train_target))

0.9615162593804117


In [16]:
## 최적 모델에 사용된 하이퍼 파라미터의 값을 출력해 확인
print(c_gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [17]:
## 각 매개변수에서 수행한 교차 검증의 평균 점수 확인
print(c_gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [18]:
## argmax함수를 이용해 점수가 가장 높은 값의 인덱스 값을 얻어
## 매개변수의 값을 확인
best_nindex = np.argmax(c_gs.cv_results_['mean_test_score'])
print(c_gs.cv_results_['params'][best_nindex])

{'min_impurity_decrease': 0.0001}


In [20]:
## 하이퍼 매개변수의 값을 하나가 아닌 여러개로 했을 때의 각 매개변수의 최적값 찾기
dict_params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
               'max_depth': range(5, 20, 1),
               'min_samples_split' : range(2, 100, 10)}

In [21]:
##  그리드서치 객체 선언 후 훈련
c_gs = GridSearchCV(DecisionTreeClassifier(random_state= 42), dict_params, n_jobs= -1)
c_gs.fit(arr_train_input, arr_train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [22]:
## 각 매개변수의 최적값 출력 후 확인
print(c_gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [23]:
## 검증 모델들의 값중 가장 큰 값 출력
print(np.max(c_gs.cv_results_['mean_test_score']))

0.8683865773302731


In [24]:
from scipy.stats import uniform, randint

rgen = randint(0,10)
rgen.rvs(10)

array([2, 1, 3, 0, 9, 0, 2, 2, 4, 3], dtype=int64)

In [25]:
np.unique(rgen.rvs(1000), return_counts= True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64),
 array([106, 103,  86, 104, 105,  93,  88, 108, 108,  99], dtype=int64))

In [26]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.98889772, 0.32046883, 0.87514496, 0.709287  , 0.07662212,
       0.29162978, 0.91019028, 0.81368123, 0.9898518 , 0.05569473])

In [27]:
## 랜덤 서치용 하이퍼 파라미터값 선언
dict_params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
               'max_depth': randint(20,50),
               'min_samples_split' : randint(2, 25),
               'min_samples_leaf' : randint(1, 25)}

In [28]:
## 랜덤 서치를 하기위해 필요한 모듈 import
from sklearn.model_selection import RandomizedSearchCV

## 랜덤 서치 객체 생성
## n_iter : 위에 선언된 하이퍼 파라미터 개수를 100개 뽑고 교차 검증 수행 후 훈련
c_gs = RandomizedSearchCV(DecisionTreeClassifier(random_state= 42), dict_params,
                          n_iter= 100, n_jobs= -1, random_state= 42)
c_gs.fit(arr_train_input, arr_train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000024E6105CE90>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000024E60E88A10>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000024E60DFA810>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000024E6105EFD0>},
                   random_state=42)

In [29]:
## 최적의 파라미터 값 출력
print(c_gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [30]:
## 검증 모델들 중 정확도가 가장 높은 값 출력
print(np.max(c_gs.cv_results_['mean_test_score']))

0.8695428296438884


In [32]:
## 최적의 모델로 결정트리를 선언하고, 테스트 세트로 성능 확인
## 성능 자체가 높은건 아니지만 검증 성능과 테스트 성능의 차이가 거의 없는것을 확인
c_dt = c_gs.best_estimator_
print(c_dt.score(arr_test_input, arr_test_target))

0.86
